In [1]:
!nvidia-smi

Mon Sep 18 02:07:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti      On | 00000000:26:00.0 Off |                  N/A |
|  0%   50C    P8               15W / 200W|     15MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q scipy
!pip install -q tensorboard
!pip install -q datasets

In [1]:
from datasets import load_dataset

#데이터를 불러옵니다
data = load_dataset("json", data_files=["train_data_clean.json"])

/home/issac/anaconda3/envs/jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset json (/home/issac/.cache/huggingface/datasets/json/default-5c4e51239604076e/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 34.59it/s]


In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'emo', 'target'],
        num_rows: 41865
    })
})

In [5]:
def makedata(x):
    return {'text': f"아래는 문장에서 대상에 대한 감정을 매우 정확하게 분류한다.\n### 문장: {x['text']}\n### 대상: {x['target']}\n### 감정: {x['emo']}"}

data = data.map(
    makedata
)

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

#모델을 불러옵니다
model_id = "EleutherAI/polyglot-ko-5.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards: 100%|██████████| 13/13 [00:29<00:00,  2.27s/it]


In [7]:
data = data.map(lambda samples: tokenizer(samples["text"], truncation=True, max_length=2048), batched=True)

In [8]:
from peft import prepare_model_for_kbit_training

#학습 자원을 위해 qlora를 사용합니다
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [9]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [10]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],
    lora_dropout=0.01, 
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 29360128 || all params: 3095846912 || trainable%: 0.9483714419532642


In [11]:
tokenizer.encode("### 감정: 기쁨")

[6, 6, 6, 3169, 29, 6716]

In [12]:
#마스킹을 위한 기준 토큰을 설정합니다.
tokenNum_emo = 3169 #감정
tokenNum_colon = 29 #:

In [13]:
!nvidia-smi

Sun Oct 22 23:20:29 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti      On | 00000000:26:00.0 Off |                  N/A |
|  0%   42C    P2               40W / 200W|   4977MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
import transformers
from transformers import Trainer
import numpy as np

class maskTrainer(Trainer):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
  
  def compute_loss(self, model, inputs, return_outputs=False):
    # print(inputs['labels'][1])
    for x in range(len(inputs['labels'])):
      maskindex = (inputs['labels'][x]==tokenNum_emo).nonzero()[:, 0]
      temp = 0
      for i, index in enumerate(maskindex):
        if (inputs['labels'][x][index+1] != tokenNum_colon):
          maskindex = np.delete(maskindex.cpu(), i-temp)
          temp += 1
      #실제 감정을 나타내는 토큰을 제외한 모든 토큰을 마스킹 처리합니다.
      inputs['labels'][x][:maskindex[0]+2] = -100
      # print(inputs['labels'][x])
        
    outputs = model(**inputs)
    loss = outputs['loss']

    return (loss,outputs) if return_outputs else loss

In [14]:
# import transformers

# # needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = maskTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        # fp16=True,
        output_dir="outputs",
        save_total_limit=2,
        logging_steps=20,
        report_to=["tensorboard"],
        num_train_epochs = 4,
        learning_rate=2e-4,
        lr_scheduler_type= "cosine",
        #optim="paged_adamw_8bit"
    
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()



You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
20,5.009900
40,2.368400
60,1.631900
80,2.065500
100,1.145100
120,1.801800
140,2.154300
160,2.083700
180,1.266200
200,1.403200


TrainOutput(global_step=1000, training_loss=1.1384563536643981, metrics={'train_runtime': 1296.2167, 'train_samples_per_second': 1.543, 'train_steps_per_second': 0.771, 'total_flos': 5589513364635648.0, 'train_loss': 1.1384563536643981, 'epoch': 0.05})

In [16]:
print("와!")

와!


In [16]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [17]:
model.save_pretrained("./saved/modu_emo_classification/polyglot-ko")

In [17]:
import numpy as np
def gen(text="", target=""):
    inputs = tokenizer(
                f"아래는 문장에서 대상에 대한 감정을 매우 정확하게 분류한다.\n### 문장: {text}\n### 대상: {target}\n### 감정:",
                return_tensors='pt',
                return_token_type_ids=False
            )

    outputs = model.generate(
            **inputs,
            max_new_tokens=1,
            temperature=0.001,
            early_stopping=True,
            return_dict_in_generate=True,
            output_scores=True
        )

    emo = [6716,1321,3065,6603,11333,4993,4921,8300]
    #여기서 emo의 토큰번호는 각각 "기쁨", "기대", "신뢰", "당황", "혐오", "공포", "분노", "슬픔"에 해당합니다.
    emo_name = ["joy","anticipation","trust","surprise","disgust","fear","anger","sadness"]
    dic = []
    for tokenNum in emo:
        outputs.sequences[0][-1] = tokenNum

        transition_scores = model.compute_transition_scores(
            outputs.sequences, (outputs.scores[0].float(),), normalize_logits=True
        )

        input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
        generated_tokens = outputs.sequences[:, input_length:]
        dic.append(round(np.exp(transition_scores[0][0].numpy())*100,2))
    return dict(zip(emo_name,dic))
    #각 감정에 대한 확률값을 반환합니다.

In [18]:
gen(text="임시완의 연기 어디까지 성장할 것인가...!! 미래가 너무 기대되는 배우입니다", target="임시완의 연기")

/home/issac/anaconda3/envs/jupyter/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/issac/anaconda3/envs/jupyter/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/issac/anaconda3/envs/jupyter/lib/python3.10/site-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on

{'joy': 52.43,
 'anticipation': 30.4,
 'trust': 16.59,
 'surprise': 0.0,
 'disgust': 0.01,
 'fear': 0.03,
 'anger': 0.01,
 'sadness': 0.05}

In [19]:
import json

def read_json(filename):
    with open(filename, "r", encoding='utf-8') as json_file:
        json_data = json.load(json_file)
    return json_data

test_data = read_json("dev_data_clean.json")

predict = []

for a, line in enumerate(test_data):
    assume = gen(text=line["text"], target=line["target"])

    predict.append(assume)
    print(a+1, assume)

with open(f"dev_data_clean_predict.json", "w", encoding='utf-8') as json_file:
    json.dump(predict, json_file, indent=2, ensure_ascii=False)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1 {'joy': 0.06, 'anticipation': 99.08, 'trust': 0.03, 'surprise': 0.01, 'disgust': 0.0, 'fear': 0.26, 'anger': 0.01, 'sadness': 0.25}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2 {'joy': 99.68, 'anticipation': 0.01, 'trust': 0.27, 'surprise': 0.01, 'disgust': 0.0, 'fear': 0.0, 'anger': 0.0, 'sadness': 0.01}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

4748 {'joy': 0.75, 'anticipation': 95.08, 'trust': 1.59, 'surprise': 1.62, 'disgust': 0.05, 'fear': 0.54, 'anger': 0.01, 'sadness': 0.02}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


4749 {'joy': 3.43, 'anticipation': 94.06, 'trust': 1.24, 'surprise': 0.01, 'disgust': 0.18, 'fear': 0.14, 'anger': 0.22, 'sadness': 0.3}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


4750 {'joy': 4.78, 'anticipation': 94.86, 'trust': 0.05, 'surprise': 0.04, 'disgust': 0.02, 'fear': 0.09, 'anger': 0.03, 'sadness': 0.05}
4751 {'joy': 89.75, 'anticipation': 9.88, 'trust': 0.26, 'surprise': 0.01, 'disgust': 0.01, 'fear': 0.01, 'anger': 0.0, 'sadness': 0.02}
